In [37]:
import re
import pymysql
import json
import pandas as pd
import csv
from pprint import pprint
from transformers import pipeline, AutoModelForSequenceClassification, BertJapaneseTokenizer
from JapaneseBPEEncoder.encode_swe import SWEEncoder_ja

In [38]:
# DB接続
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='web_app',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
connection.commit()
cur = connection.cursor()

In [39]:
APPID = 582010  # 582010：モンハンワールド
ROWNUM = 100

In [55]:
'''
cur.execute(' \
    select \
        recommendationid, \
        appid, \
        review as text, \
        voted_up  \
    from \
        review \
')

'''
cur.execute(' \
    select \
        recommendationid, \
        appid, \
        review as text, \
        voted_up  \
    from \
        review \
    where \
        appid = {0} \
'.format(APPID))

"\ncur.execute('     select         recommendationid,         appid,         review as text,         voted_up      from         review     where         appid = {0} '.format(APPID))\n"

In [19]:
# パイプラインの準備
model = AutoModelForSequenceClassification.from_pretrained('daigo/bert-base-japanese-sentiment') 
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
nlp = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)

# エンコーダー
with open('JapaneseBPEEncoder/ja-swe32k.txt', encoding='utf-8') as f:
    bpe = f.read().split('\n')
with open('JapaneseBPEEncoder/emoji.json', encoding='utf-8') as f:
    emoji = json.loads(f.read())
enc = SWEEncoder_ja(bpe, emoji)



# 感情分析の実行
result = {
    'recommendationid' : [],
    'label' : [],
    'score' : [],
    'result' : [],
    'text' : []
}
fetch_cnt = 0
data_cnt = 0
correct_cnt = 0
FORMAT = '{0}\t{1}\t{2}\t{3}\t{4}\n'
datas = cur.fetchmany(ROWNUM)
fetch_cnt += 1
fw = open('Predict/' + str(APPID) + '.tsv', 'w', encoding='utf-8')
fw.write(FORMAT.format('recommendationid', 'label', 'score', 'correct', 'text'))
while len(datas) > 0:
    for index, data in enumerate(datas):
        if re.match(r'[ぁ-んァ-ン一-龥]+', data['text']) and len(data['text']) > 48:
            text = enc.clean_text(data['text'])[:256]
            senti = nlp(text)
            label_flg = 1 if senti[0]['label'] == 'ポジティブ' else 0
            if label_flg == data['voted_up']:
                correct = '正解'
                correct_cnt += 1
            else:
                correct = '不正解'
            fw.write(FORMAT.format(data['recommendationid'], senti[0]['label'], senti[0]['score'], correct, text.replace('\n','\\n')))

            '''
            result['recommendationid'].insert(index % 10, data['recommendationid'])
            result['label'].insert(index % 10, senti[0]['label'].ljust(10)) 
            result['score'].insert(index % 10, senti[0]['score']) 
            result['result'].insert(index % 10, correct.ljust(6))
            result['text'].insert(index % 10, text.ljust(256))

            if (index % 10 == 9):
                df = pd.DataFrame(result)
                print(df)
            '''
            data_cnt += 1
    datas = cur.fetchmany(ROWNUM)
    fetch_cnt += 1
            
print('正解率：{0}%、データ件数：{1}件'.format(correct_cnt / data_cnt * 100, data_cnt))
        
fw.close()  
f.close()      

正解率：68.99441340782123%、データ件数：1074件


In [31]:
cur.execute(' \
    select \
        recommendationid, \
        appid, \
        review as text, \
        voted_up  \
    from \
        review \
    where \
        appid = 582010 \
    and \
        recommendationid = 44243017 \
')
review = cur.fetchone()
pprint(review)

{'appid': 582010,
 'recommendationid': 44243017,
 'text': 'いいところ\n'
         '楽しい\n'
         '60fpsでモンハンできる\n'
         'グラがいい\n'
         '採取が楽になった\n'
         '\n'
         'わるいところ\n'
         '[strike]ストーリー期待してたけど予想より短い\n'
         '修正パッチ予告してるけど今のところネットワークエラー多すぎ\n'
         'テオのようなエフェクト多いモンスターと戦うとfpsガク落ちする[/strike] \n'
         '修正済み\n'
         '\n'
         'アイスボーンも出るし今からやっても間に合うぞ！\n'
         'アイボーやらないか？',
 'voted_up': 1}


In [32]:
## テスト実行
# 元文章
print(' Original: ', review['text'])
# Tokenizer
print('Tokenized: ', tokenizer.tokenize(review['text']))
# Token-id
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(review['text'])))

 Original:  いいところ
楽しい
60fpsでモンハンできる
グラがいい
採取が楽になった

わるいところ
[strike]ストーリー期待してたけど予想より短い
修正パッチ予告してるけど今のところネットワークエラー多すぎ
テオのようなエフェクト多いモンスターと戦うとfpsガク落ちする[/strike] 
修正済み

アイスボーンも出るし今からやっても間に合うぞ！
アイボーやらないか？
Tokenized:  ['いい', 'ところ', '楽しい', '60', 'f', '##ps', 'で', 'モン', '##ハン', 'できる', 'グラ', 'が', 'いい', '採取', 'が', '楽', 'に', 'なっ', 'た', 'わ', '##る', '##い', 'ところ', '[', 'st', '##ri', '##ke', ']', 'ストーリー', '期待', 'し', 'て', 'た', 'けど', '予想', 'より', '短い', '修正', 'パッチ', '予告', 'し', 'てる', 'けど', '今', 'の', 'ところ', 'ネットワーク', 'エラー', '多', 'すぎ', 'テオ', 'の', 'よう', 'な', 'エフェクト', '多い', 'モンスター', 'と', '戦う', 'と', 'f', '##ps', 'ガ', '##ク', '落ち', 'する', '[', '##/', 'st', '##ri', '##ke', ']', '修正', '済み', 'アイス', '##ボーン', 'も', '出る', 'し', '今', 'から', 'やっ', 'て', 'も', '間に', '##合う', 'ぞ', '!', 'アイ', '##ボー', 'やら', 'ない', 'か', '?']
Token IDs:  [2575, 1134, 19835, 1840, 1044, 9893, 12, 2008, 2793, 392, 1984, 14, 2575, 12126, 14, 820, 7, 58, 10, 630, 28449, 28457, 1134, 4314, 4149, 2879, 8458, 4118, 3335, 3252, 15, 16, 10, 11218, 4663, 221, 5